# RDT Kernel Demo

This notebook demonstrates the basic usage of the RDT (Recursive Diffusion Transform) Kernel.

The RDT Kernel evolves a 2D scalar field according to:

$$\frac{\partial L}{\partial t} = -\alpha \ln(L) + D \nabla^2 L$$

In [ ]:
import torch
import math
import matplotlib.pyplot as plt
from rdt_kernel import get_device, rdt_kernel, step, run_demo

## Quick Demo

Run a simple simulation and see the performance:

In [ ]:
# Run a 128x128 simulation for 100 steps
L = run_demo(n=128, steps=100, alpha=0.5, D=0.1, dx=1.0, dt=0.01)

## Custom Simulation with Visualization

In [ ]:
# Setup
device, name = get_device()
print(f"Using: {name}")

n = 256
alpha = 0.5
D = 0.1
dx = 1.0
dt = 0.01

# Initialize field with sinusoidal pattern
x = torch.linspace(0, 2*math.pi, n, device=device).unsqueeze(1)
y = torch.linspace(0, 2*math.pi, n, device=device).unsqueeze(0)
L = 1.0 + 0.1 * torch.sin(x) * torch.cos(y)

In [ ]:
# Visualize initial state
plt.figure(figsize=(6, 5))
plt.imshow(L.cpu().numpy(), cmap='viridis', origin='lower')
plt.colorbar(label='Field Value')
plt.title('Initial State')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
# Run simulation and track statistics
steps = 100
means = []
variances = []

for i in range(steps):
    means.append(L.mean().item())
    variances.append(L.var().item())
    L = step(L, alpha, D, dx, dt)

print(f"Final mean: {L.mean().item():.6f}")
print(f"Final variance: {L.var().item():.6f}")

In [ ]:
# Visualize final state
plt.figure(figsize=(6, 5))
plt.imshow(L.cpu().numpy(), cmap='viridis', origin='lower')
plt.colorbar(label='Field Value')
plt.title(f'Final State (after {steps} steps)')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
# Plot statistics over time
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(means)
ax1.set_xlabel('Step')
ax1.set_ylabel('Mean Field Value')
ax1.set_title('Mean Evolution')
ax1.grid(True, alpha=0.3)

ax2.plot(variances)
ax2.set_xlabel('Step')
ax2.set_ylabel('Variance')
ax2.set_title('Variance Decay (Self-Stabilization)')
ax2.set_yscale('log')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Experiment with Different Parameters

Try changing:
- `alpha`: Damping coefficient (higher = more damping)
- `D`: Diffusion constant (higher = more spreading)
- `dt`: Time step (larger = faster evolution, but may activate clamping)

In [ ]:
# Compare different time steps
time_steps = [0.005, 0.01, 0.02]
colors = ['blue', 'green', 'red']

plt.figure(figsize=(10, 6))

for dt_test, color in zip(time_steps, colors):
    L_test = 1.0 + 0.1 * torch.sin(torch.linspace(0, 2*math.pi, 256, device=device)).unsqueeze(1)
    vars_test = []
    
    for _ in range(200):
        vars_test.append(L_test.var().item())
        L_test = step(L_test, alpha=0.5, D=0.1, dx=1.0, dt=dt_test)
    
    plt.plot(vars_test, color=color, label=f'dt={dt_test}', linewidth=2)

plt.yscale('log')
plt.xlabel('Step')
plt.ylabel('Variance (log scale)')
plt.title('Effect of Time Step on Stability')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Observations

- **Smaller dt** (0.005): Smooth, gradual evolution
- **Medium dt** (0.01): Strong damping with stable dynamics
- **Larger dt** (0.02): Rapid stabilization via increased clamp activity

The kernel demonstrates **self-stabilizing behavior** across all parameter ranges!